In [8]:
'''
The leg is structured as follows:
(trip_id, departure_node, departure_time, arrival_node, arrival_time, route_id, service_id)

trip_id = leg.iloc[0]
departure_node = leg[1]
departure_time = leg[2]
arrival_node = leg[3]
arrival_time = leg[4]
route_id = leg[5]
service_id = leg[6]
'''

'\nThe leg is structured as follows:\n(trip_id, departure_node, departure_time, arrival_node, arrival_time, route_id, service_id)\n\ntrip_id = leg.iloc[0]\ndeparture_node = leg[1]\ndeparture_time = leg[2]\narrival_node = leg[3]\narrival_time = leg[4]\nroute_id = leg[5]\nservice_id = leg[6]\n'

In [9]:
from datetime import datetime, timedelta
import pandas as pd
import math
import scipy.stats as stats
from data_preparation import prepare_data,import_data
pd.set_option('display.max_colwidth', None)

In [10]:
agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df,calendar_dates_df = import_data()
legs_df = prepare_data(stops_df,trips_df,stop_times_df)

In [11]:
print(legs_df.head())

0               (1.TA.1-MB4-j24-1.1.R, Schruns Bahnhof, 07:01:00, Tschagguns Bahnhof, 07:02:00, 1-MB4-j24-1, TA+m7030)
1         (1.TA.1-MB4-j24-1.1.R, Tschagguns Bahnhof, 07:02:00, Kaltenbrunnen Bahnhof, 07:04:00, 1-MB4-j24-1, TA+m7030)
2            (1.TA.1-MB4-j24-1.1.R, Kaltenbrunnen Bahnhof, 07:04:00, Vandans Bahnhof, 07:07:00, 1-MB4-j24-1, TA+m7030)
3    (1.TA.1-MB4-j24-1.1.R, Vandans Bahnhof, 07:07:00, St. Anton im Montafon Bahnhof, 07:10:00, 1-MB4-j24-1, TA+m7030)
4     (1.TA.1-MB4-j24-1.1.R, St. Anton im Montafon Bahnhof, 07:11:00, Lorüns Bahnhof, 07:13:00, 1-MB4-j24-1, TA+m7030)
Name: leg, dtype: object


In [12]:
# Function to calculate probability of successful transfer between two subsequent legs
def calculate_transfer_probability(prev_leg: pd.Series, next_leg: pd.Series) -> float:
    if prev_leg[0] == next_leg[0]:
        return 1
    else:
        prev_arrival_time = datetime.strptime(prev_leg[4], "%H:%M:%S")
        next_departure_time = datetime.strptime(next_leg[2], "%H:%M:%S")
        
        # Calculate transfer time in minutes
        transfer_time = (next_departure_time - prev_arrival_time).total_seconds() / 60
        if transfer_time < 2:
            return 0
        #If tripId = next tripId , then probability = 1 since the traveler remains on the same line
        else:
            return stats.gamma.cdf(transfer_time, a=2, scale=2)
            #return 0.5


In [13]:
# i hope it is cumulative distribution
def calculate_cumulative_probability(itinerary) -> list[float]:
    cumulative_probabilities = [1] 
    for i in range(len(itinerary) - 1):
        prev_leg = itinerary[i]
        next_leg = itinerary[i+1]

        transfer_prob = calculate_transfer_probability(prev_leg, next_leg)
        cumulative_probabilities.append(transfer_prob)
    return cumulative_probabilities
  

In [14]:
# if there is a chance to have all succesful transfers, is the arrival time in time budget
def calculate_arrival_probability(itinerary, start_time, time_budget) -> int:
    # Check if all previous transfers were successful
    if math.prod(calculate_cumulative_probability(itinerary)) > 0:  # Only proceed if all transfers are successful( naka can be possibly made)
        # Calculate the actual arrival time at the final leg
        destination_leg = itinerary[-1]
        destination_arrival_time = destination_leg[4]
        
        start_time = datetime.strptime(start_time, "%H:%M:%S")
        destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

        total_travel_time = destination_arrival_time - start_time

        # Check if total travel time is within the budget
        if total_travel_time <= time_budget:
            return 1  # Probability of 1 if arrival is within the time budget
        else:
            return 0  # Probability of 0 if arrival is beyond the time budget
    else:
        return 0  # Probability of 0 if any prior transfer was unsuccessful

In [15]:
def primary_itinerary_reliability(itinerary, start_time, time_budget) -> float:
    
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    arrival_probability = calculate_arrival_probability(itinerary,start_time,time_budget)

    reliability = arrival_probability * product_of_probabilities
    return reliability

To change this

In [16]:
#im not sure how the "backup" will look like (like is it starting from the origin or from the missed transfer point)
# backup = (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
#backup_legs = [list of backups excluding prim]
def backup_itinerary_reliability(itinerary, backup:tuple, start_time, time_budget) -> float:
    
    #change the format
    backup_itinerary = backup[0]
    arrival_probability = calculate_arrival_probability(backup_itinerary, start_time, time_budget)
    cumulative_probabilities = calculate_cumulative_probability(backup_itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    
    #some adj
    #primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)

    initial_transfer_prob = 1 # Default value in case no missed transfer is identified
    
    #but what if backup not to the last leg????
    for idx, leg in enumerate(itinerary[:-1]):  # Exclude last item since we're accessing i+1
        #basically comparing if it is the same leg
        if leg[3] == backup_itinerary[0][1]:
            prev_leg = itinerary[idx]
            missed_leg = itinerary[idx + 1]
            initial_transfer_prob = calculate_transfer_probability(prev_leg,missed_leg)
            backup_transfer_prob = calculate_transfer_probability(prev_leg,backup_itinerary[0])
            #print("init_tr",initial_transfer_prob)
            break
    
    print((1-initial_transfer_prob))
    print(backup_transfer_prob)
    print(product_of_probabilities)
    
    backup_reliability = arrival_probability * product_of_probabilities * min((1 - initial_transfer_prob),backup_transfer_prob)
    print("b_rel",backup_reliability)
    return backup_reliability


In [17]:
#Reliability of a complete itinerary 
#Backups will be set of backups of prim itineary
# backup itineary= (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
def itinerary_reliability(itinerary : list[pd.Series],Backups : list[tuple], start_time:str, time_budget: timedelta) -> float:
    primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
    #print("prim",primary_reliability)
    if primary_reliability > 0 :
        added_reliability = 0
        for backup,reliability in Backups:
            #this will be fucked up, as we dont know yet how to make backups ( like on what format)
            backup_reliability = backup_itinerary_reliability(itinerary,backup,start_time,time_budget)
            added_reliability += backup_reliability
        #added_reliability = min(added_reliability, 1-primary_reliability)
        complete_reliability = primary_reliability + added_reliability
        return complete_reliability
    else:
        return 0.0

In [18]:
Bak = [(([('72.TA.10-A12-j24-1.18.R', 'Wiener Neustadt Hauptbahnhof', '15:30:00', 'Wien Meidling', '15:55:00', '10-A12-j24-1', 'TA+n4'), ('72.TA.10-A12-j24-1.18.R', 'Wien Meidling', '15:57:00', 'Wien Hauptbahnhof', '16:02:00', '10-A12-j24-1', 'TA+n4'), ('72.TA.10-A12-j24-1.18.R', 'Wien Hauptbahnhof', '16:12:00', 'Flughafen Wien Bahnhof', '16:27:00', '10-A12-j24-1', 'TA+n4')], timedelta(seconds=7020)), 0.13588822540043322), 
       (([('65.TA.4-CX9-W-j24-1.31.R', 'Wien Meidling', '16:04:00', 'Wien Hauptbahnhof', '16:08:00', '4-CX9-W-j24-1', 'TA+o2'), ('72.TA.10-A12-j24-1.18.R', 'Wien Hauptbahnhof', '16:12:00', 'Flughafen Wien Bahnhof', '16:27:00', '10-A12-j24-1', 'TA+n4')], timedelta(seconds=7020)), 0.2411650997112305), 
       (([('72.TA.10-A12-j24-1.18.R', 'Wien Hauptbahnhof', '16:12:00', 'Flughafen Wien Bahnhof', '16:27:00', '10-A12-j24-1', 'TA+n4')], timedelta(seconds=7020)), 0.1991482734714558)]

In [19]:
a = [('150.TA.20-SV5-L-j24-1.13.H', 'Schattendorf Kirchengasse', '14:43:00', 'Loipersbach-Schattendorf Bhf.', '14:48:00', '20-SV5-L-j24-1', 'TA+byk20'), ('42.TA.3-R93-A-j24-1.7.R', 'Loipersbach-Schattendorf Bhf.', '14:54:00', 'Marz-Rohrbach Bahnhof', '14:58:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Marz-Rohrbach Bahnhof', '14:59:00', 'Mattersburg Bahnhof', '15:01:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Mattersburg Bahnhof', '15:02:00', 'Mattersburg Nord Bahnhof', '15:03:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Mattersburg Nord Bahnhof', '15:04:00', 'Wiesen-Sigleß Bahnhof', '15:07:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Wiesen-Sigleß Bahnhof', '15:07:00', 'Bad Sauerbrunn Bahnhof', '15:11:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Bad Sauerbrunn Bahnhof', '15:13:00', 'Neudörfl Bahnhof', '15:16:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Neudörfl Bahnhof', '15:17:00', 'Katzelsdorf Bahnhof', '15:20:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Katzelsdorf Bahnhof', '15:20:00', 'Wiener Neustadt Hauptbahnhof', '15:25:00', '3-R93-A-j24-1', 'TA+3j030'), ('30.TA.10-A12-j24-1.7.R', 'Wiener Neustadt Hauptbahnhof', '15:27:00', 'Wien Meidling', '15:55:00', '10-A12-j24-1', 'TA+o2p10'), ('45.TA.2-RX2-W-j24-1.27.R', 'Wien Meidling', '15:59:00', 'Wien Matzleinsdorfer Platz', '16:01:00', '2-RX2-W-j24-1', 'TA+n4'), ('45.TA.2-RX2-W-j24-1.27.R', 'Wien Matzleinsdorfer Platz', '16:02:00', 'Wien Hauptbahnhof', '16:03:00', '2-RX2-W-j24-1', 'TA+n4'), ('124.TA.10-A3-j24-1.29.H', 'Wien Hauptbahnhof', '16:09:00', 'Flughafen Wien Bahnhof', '16:24:00', '10-A3-j24-1', 'TA+7m520')]
res = itinerary_reliability(a,Bak,"14:30:00", timedelta(hours=2,minutes=10))
print(res)

0.7357588823428847
0.7127025048163542
1
b_rel 0.7127025048163542
0.40600584970983844
0.9389005190396673
0.5939941502901616
b_rel 0.2411650997112305
0.1991482734714558
0.9389005190396673
1
b_rel 0.1991482734714558
1.2536828019103206


In [20]:
def get_available_service_ids(start_date):
    # Convert start_date to datetime and then to integer-like string (YYYYMMDD) without dashes
    start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    start_date_str = start_date_datetime.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_date_datetime.weekday()]

    available_service_ids = []

    # Step 1: Check for exceptions in calendar_dates
    exceptions = calendar_dates_df[calendar_dates_df["date"] == start_date_str]
    # Go through each service_id to check for exceptions
    for _, exception in exceptions.iterrows():
        service_id = exception["service_id"]
        if exception["exception_type"] == 2:  # Service is added as an exception
            available_service_ids.append(service_id)
        elif exception["exception_type"] == 1:  # Service is removed as an exception
            if service_id in available_service_ids:
                available_service_ids.remove(service_id)

    # Step 2: Check for regular service in calendar
    for _, service in calendar_df.iterrows():
        service_id = service["service_id"]
        # Check if date is within the start_date and end_date range
        if int(service["start_date"]) <= int(start_date_str) <= int(service["end_date"]):
            # Check if the service operates on this weekday
            if service[weekday] == 1:
                if service_id not in available_service_ids:
                    available_service_ids.append(service_id)

    return available_service_ids

In [21]:
def filter_network(start_time,start_date,time_budget):
    available_sevices = get_available_service_ids(start_date)

    filtered_network= []
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    end_time = start_time + time_budget
    print(end_time)
    for row in legs_df:
        leg_departure_time = datetime.strptime(row[2],"%H:%M:%S")
        leg_arrival_time = datetime.strptime(row[4],"%H:%M:%S")
        if leg_departure_time >= start_time and leg_departure_time<= end_time and leg_arrival_time <= end_time:
            service_id = row[6]
            if service_id in available_sevices:
                filtered_network.append(row)
    print("Network",len(filtered_network))
    return filtered_network

In [22]:
#not sure if here we should add time dependancy
def search_adjecent_legs(node_id,arrival_time,filtered_legs):
    arrival_time = datetime.strptime(arrival_time, "%H:%M:%S")
    adjecent_legs = []
    for i in range(0,len(filtered_legs)):
        leg = filtered_legs[i]
        leg_departure_time = datetime.strptime(leg[2],"%H:%M:%S")
        if leg[1] == node_id and leg_departure_time >= arrival_time :
            adjecent_legs.append(leg)
    #print("ADJ",len(adjecent_legs))
    return adjecent_legs

In [23]:
# need to add to arrival probability
def travel_time(itinerary, start_time):
    destination_leg = itinerary[-1]
    destination_arrival_time = destination_leg[4]
        
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

    total_travel_time = destination_arrival_time - start_time
    return total_travel_time

In [24]:
def is_transfer(itinerary) -> bool:
    prev_leg = itinerary[-2]
    next_leg = itinerary[-1]
    if prev_leg[0] == next_leg[0]:
        return False
    else:
        return True

In [25]:
def find_min_index(LIST:list):
    """Find the index of the itinerary with the shortest duration."""
    #adjust for bachup
    min_index = 0
    min_value =  LIST[0][1] 
    for i in range(1, len(LIST)):
        current_value = LIST[i][1]
        if current_value < min_value:
            min_value = current_value
            min_index = i

    return min_index

In [26]:
def update_visited_stops(visited_stops, leg):
    """Update the visited stops dictionary with a new leg."""
    destination_stop = leg[1]
    arrival_time = datetime.strptime(leg[4], "%H:%M:%S")
    trip_id = leg[0]
    visited_stops[destination_stop] = (arrival_time, trip_id)

In [27]:
def check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget):
    """Check and update the most reliable path (MRIB) if applicable."""
    Backups = shortest_path[4][:]
    rel = itinerary_reliability(shortest_path[0], Backups, start_time, time_budget)
    if rel > MRIB_reliability:
        print("New most reliable path found")
        MRIB_reliability = rel
        MRIB = shortest_path
    return MRIB_reliability, MRIB

In [28]:
def can_visit_leg(leg, visited_stops, shortest_path_trip_id, destination_node):


    arrival_stop = leg[3]  # Get destination stop ID
    arrival_time = datetime.strptime(leg[4], "%H:%M:%S")  # Arrival time
    trip_id = leg[0]  # Assuming trip_id is at index 0
    if destination_node != arrival_stop:
        if arrival_stop in visited_stops:
            visited_time, visited_trip_id = visited_stops[arrival_stop]
            if trip_id == shortest_path_trip_id:
                if arrival_time <= visited_time:
                    visited_stops[arrival_stop] = (arrival_time, trip_id)
                return True
            else : 
                if arrival_time > visited_time:
                    return False
                elif arrival_time == visited_time and trip_id == visited_trip_id:
                    return True
                elif arrival_time == visited_time:
                    return False
                else:
                    visited_stops[arrival_stop] = (arrival_time, trip_id)
                    return True

        else:
            visited_stops[arrival_stop] = (arrival_time, trip_id)
            return True
    return True

In [29]:
def origin_node_filtering(origin_node: int, start_time: str, time_budget: timedelta, legs: list):
    """
    Filters legs to ensure departure from the origin occurs within the first 25% of the time budget.

    Parameters:
    origin_node (int): The origin stop/node.
    start_time (str): The starting time in "%H:%M:%S" format.
    time_budget (timedelta): The total allowed time budget.
    legs (list): The list of available legs to filter.

    Returns:
    list: Filtered legs meeting the departure condition.
    """
    
    # Calculate the 20% time limit from the time budget
    max_departure_time = datetime.strptime(start_time, "%H:%M:%S") + time_budget * 0.15

    # Filter legs based on departure time
    filtered_legs = []
    for leg in legs:
        departure_time = datetime.strptime(leg[2], "%H:%M:%S")  # Assuming departure time is stored in this location
        if departure_time <= max_departure_time:
            filtered_legs.append(leg)

    return filtered_legs


In [30]:
def transform_route_info(MRIB,MRIB_reliability,Backups):
    primary_itinerary = MRIB  # Primary itinerary (first item in best_result_fast)
    reliability = MRIB_reliability  # Reliability score
    duration = MRIB[1]  # Duration of the trip
    arrival_time = primary_itinerary[-1][4]  # Arrival time

    
    # Initialize the grouped_routes list
    grouped_routes = []
    last_route = None  # To track the previous route for grouping

    # Step 1: Process the primary itinerary
    for i in range(len(primary_itinerary)):
        current_stop = primary_itinerary[i][1]
        route_id = primary_itinerary[i][5]
        departure_time = primary_itinerary[i][2]
        arrival_time = primary_itinerary[i][4]
        next_stop = primary_itinerary[i][3]

        # Grouping the routes based on the route ID
        if route_id == last_route:
            grouped_routes[-1]["stops"].append((next_stop, arrival_time))
        else:
            grouped_routes.append({
                "route_id": route_id,
                "start_stop": current_stop,
                "departure_time": departure_time,
                "stops": [(next_stop, arrival_time)]
            })
        last_route = route_id

    # Step 2: Print the grouped routes for the primary itinerary
    for segment in grouped_routes:
        start = segment["start_stop"]
        dep_time = segment["departure_time"]
        route = segment["route_id"]
        stops = " → ".join([f"{stop} (Ankunft: {arr})" for stop, arr in segment["stops"]])
        print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

    # Step 3: Print additional details
    print(f"\n🎯 Endstation: {primary_itinerary[-1][3]} (Ankunft: {arrival_time})")
    print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability:.2f}\n")
    
    # Step 4: Process and print the backup routes
    if Backups:
        print("🔄 Backups:")
        for backup in Backups:
            backup_path = backup[0][0]
            backup_reliability = backup[1]
            grouped_backup_routes = []
            last_backup_route = None
            transfer_point = backup_path[0][1]

            print(f"\nBackup from {transfer_point} :")
            for i in range(len(backup_path)):
                current_stop = backup_path[i][1]
                route_id = backup_path[i][5]
                departure_time = backup_path[i][2]
                arrival_time = backup_path[i][4]
                next_stop = backup_path[i][3]

                # Group backup routes by route ID
                if route_id == last_backup_route:
                    grouped_backup_routes[-1]["stops"].append((next_stop, arrival_time))
                else:
                    grouped_backup_routes.append({
                        "route_id": route_id,
                        "start_stop": current_stop,
                        "departure_time": departure_time,
                        "stops": [(next_stop, arrival_time)]
                    })
                last_backup_route = route_id

            # Print backup route segments
            for segment in grouped_backup_routes:
                start = segment["start_stop"]
                dep_time = segment["departure_time"]
                route = segment["route_id"]
                stops = " → ".join([f"{stop} (Ankunft: {arr})" for stop, arr in segment["stops"]])
                print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

            print(f"🔹 Gesamt-Zuverlässigkeit der Backup-Route: {backup_reliability:.2f}\n")
            


In [40]:
def find_primary_path(origin_node: int,destination_node: int, start_time : str, time_budget: timedelta, filtered_legs):
    
    '''Initial setup'''
    MRIB_reliability = 0
    LISTofTRIPS = []
    LISTofCompletedTRIPS = []
    visited_stops = {}
    n = 0

    visited_stops[origin_node] = (datetime.strptime(start_time, "%H:%M:%S"), "")


    ''' Search initial adjacent legs'''
    adjecent_legs = search_adjecent_legs(origin_node,start_time,filtered_legs)
    filtered_adj_legs = origin_node_filtering(origin_node,start_time,time_budget,adjecent_legs)
    for leg in filtered_adj_legs:
        itinerary  = [leg]
        duration = travel_time(itinerary,start_time)
        reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
        if reliability > 0 and timedelta(seconds=0) < duration <= time_budget:
            LISTofTRIPS.append([itinerary,duration])
        update_visited_stops(visited_stops, leg)
    #print("visited stops", visited_stops)
    #print(len(LISTofTRIPS))
    
    '''Main Loop'''
    while LISTofTRIPS:
        n +=1 #just count
        print("New iteration", n)

        visited_stops_n = visited_stops.copy()
        passed_stops_n = []

        # Find the shortest itinerary(no, min function is not working)
        min_index = find_min_index(LISTofTRIPS)
        shortest_path = LISTofTRIPS.pop(min_index)
        #print("Shortest_path",shortest_path)

        tail = shortest_path[0][-1] # last leg of trip

        # Check if destination is reached
        if tail[3] == destination_node:
            # list of completed trips
            #reliability = primary_itinerary_reliability(shortest_path[0],start_time,time_budget)
            #if reliability >= MRIB_reliability:
            LISTofCompletedTRIPS.append(shortest_path)
            continue

        for leg in shortest_path[0]:
            update_visited_stops(visited_stops_n, leg)
            destination_stop = leg[3]
            passed_stops_n.append(destination_stop)
        
        # Explore connections from the current tail
        #print("Search for the next connection")
        #LIST_NEXT = []
        #set of all legs adjacent to tail leg m
        next_legs = search_adjecent_legs(tail[3],tail[4],filtered_legs)
        #that was some idea, but bad idea
        #next_legs_filtered = filter_adjecent_legs(next_legs,tail.iloc[0])
        #another idea of keeping track of visited nodes

        # no passed stops!
        next_legs = [leg for leg in next_legs if leg[3] not in passed_stops_n]

        next_legs = [
            leg for leg in next_legs 
            if leg[0] == tail[0] or 
            datetime.strptime(leg[2], "%H:%M:%S") >= (datetime.strptime(tail[4], "%H:%M:%S") + timedelta(minutes=2)) 
             #and datetime.strptime(leg[2], "%H:%M:%S") <=(datetime.strptime(tail[4], "%H:%M:%S") + timedelta(minutes=30)))
        ]
        
        #print("before filter",len(next_legs))
        next_legs = [
            leg for leg in next_legs 
            if can_visit_leg(leg, visited_stops_n, shortest_path[0][-1][0],destination_node)
        ]
        #print("1st filter",len(next_legs))
        next_legs = [
                leg for leg in next_legs 
                if can_visit_leg(leg, visited_stops_n,shortest_path[0][-1][0],destination_node)
            ]
        #print("2nd filter",len(next_legs))
        
        for leg in next_legs:
            itinerary = shortest_path[0] + [leg] #combine previous legs and adjecent
            duration = travel_time(itinerary,start_time)
            reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
            if reliability > 0 and timedelta(seconds=0) < duration <= time_budget:
                LISTofTRIPS.append([itinerary,duration])
            #update_visited_stops(visited_stops_n, leg)
    print("Primary_path_found")
    return LISTofCompletedTRIPS, visited_stops
       

In [41]:
def backup_search(shortest_next_itinerary,destination_node,start_time,time_budget,filtered_legs,visited_stops):

    #Initial Setup
    MRB_reliability = 0
    MRB = None
    LIST_Backups = []
    
    visited_stops_b = visited_stops.copy()
    transfer_leg = shortest_next_itinerary[-2]
    primary_itinerary = shortest_next_itinerary[:-1]
    missed_leg_departure = datetime.strptime(shortest_next_itinerary[-1][2], "%H:%M:%S")

    passed_stops_b =[]
    for leg in primary_itinerary:
        update_visited_stops(visited_stops_b,leg)
        destination_stop = leg[3]
        passed_stops_b.append(destination_stop)

    transfer_point = transfer_leg[3]


    adjecent_legs = search_adjecent_legs(transfer_point,primary_itinerary[-1][2],filtered_legs)

    adjecent_legs = [leg for leg in adjecent_legs if leg[3] not in passed_stops_b]

    adjecent_legs = [
            leg for leg in adjecent_legs 
            if datetime.strptime(leg[2], "%H:%M:%S") >= (missed_leg_departure + timedelta(minutes=2)) 
        ]

    adjecent_legs = [
        leg for leg in adjecent_legs 
        if can_visit_leg(leg, visited_stops_b, "",destination_node)
    ]
            
    adjecent_legs = [
        leg for leg in adjecent_legs 
        if can_visit_leg(leg, visited_stops_b,"",destination_node)
    ]


    for leg in adjecent_legs:
        backup_legs = [leg]
        b_duration = travel_time(backup_legs,start_time)
        backup_full = (backup_legs,b_duration)
        # maybe compare to MRIB rel
        if timedelta(seconds=0) < b_duration <= time_budget:
            LIST_Backups.append(backup_full)
            
        #update_visited_stops(visited_stops_b, b_tail)
    

    #Backup loop
    while len(LIST_Backups) > 0:  
        min_index_b = 0
        min_value_b =  LIST_Backups[0][1]# durarion 
        for i in range(1, len(LIST_Backups)):
            current_value_b = LIST_Backups[i][1]
            if current_value_b < min_value_b:
                min_value_b = current_value_b
                min_index_b = i
    	
        shortest_backup = LIST_Backups.pop(min_index_b)
        #print("Shortest backup",shortest_backup)
        
        
        for leg in shortest_backup[0]:
            destination_stop = leg[3]
            passed_stops_b.append(destination_stop)

        b_tail = shortest_backup[0][-1]

        if b_tail[3] == destination_node:
            rel = backup_itinerary_reliability(shortest_next_itinerary,shortest_backup,start_time,time_budget)
            if rel > MRB_reliability:
                MRB_reliability = rel
                MRB = shortest_backup
                #print("dest reached, MRB upd",MRB_reliability)
        else:
            
            next_legs_b = search_adjecent_legs(b_tail[3],b_tail[4],filtered_legs)
            
            next_legs_b = [leg for leg in next_legs_b if leg[3] not in passed_stops_b]
            
            next_legs_b = [
            leg for leg in next_legs_b 
            if leg[0] == b_tail[0] or datetime.strptime(leg[2], "%H:%M:%S") >= (datetime.strptime(b_tail[4], "%H:%M:%S") + timedelta(minutes=2))
        ]
            next_legs_b = [
                leg for leg in next_legs_b 
                if can_visit_leg(leg, visited_stops_b, shortest_backup[0][-1][0],destination_node)
            ]
            
            next_legs_b = [
                leg for leg in next_legs_b 
                if can_visit_leg(leg, visited_stops_b, shortest_backup[0][-1][0],destination_node)
            ]


            for leg in next_legs_b:
                backup_legs = shortest_backup[0][:]
                backup_legs.append(leg)
                b_duration = travel_time(backup_legs,start_time)
                backup_full = (backup_legs,b_duration)
                b_reliability = backup_itinerary_reliability(shortest_next_itinerary,backup_full,start_time,time_budget)
                if timedelta(seconds=0) < b_duration <= time_budget:
                    LIST_Backups.append(backup_full)
                   
                #else:
                    #print("Backup Reliability 0 -2")
    
    # end of while len(LIST_Backups) > 0 loop       
    #print("MRB", MRB)
    return MRB,MRB_reliability
                 
            
# end of while len(LIST_Backups) > 0 loop     

In [42]:
def find_path(origin_node,destination_node,start_datetime,time_budget):
    MRIB_reliability = 0
    MRIB = None
    start_date, start_time = start_datetime.split()
    transfer_dict = {}
    filtered_legs = filter_network(start_time,start_date,time_budget)
    completed_primary_trips, visited_stops = find_primary_path(origin_node,destination_node,start_time,time_budget,filtered_legs)
    
    for trips in completed_primary_trips:
        primary_itinerary = trips[0]  # Extract the primary itinerary
        growing_itinerary = []  # Start with an empty list for the growing itinerary
        Backups = []
        
        print("Prim_it", primary_itinerary)
        for leg in primary_itinerary:
            growing_itinerary.append(leg)  # Add the current leg to the growing itinerary
            
            # Only start checking for transfers once there are at least two legs
            if len(growing_itinerary) > 1:

                if is_transfer(growing_itinerary):
                    transfer = (
                        growing_itinerary[-2][3],
                        growing_itinerary[-2][4],
                        growing_itinerary[-1][1],
                        growing_itinerary[-1][2])
                    
                    MRB_reliability = 0
                    MRB = None
                    if transfer not in transfer_dict.keys():
                        MRB, MRB_reliability = backup_search(growing_itinerary,destination_node,start_time,time_budget,filtered_legs,visited_stops)
                        transfer_dict[transfer] = MRB,MRB_reliability
                    else:
                        MRB,MRB_reliability = transfer_dict.get(transfer)
                    print("MRB",MRB)
                    if MRB: 
                        Backups.append((MRB,MRB_reliability))

        reliability = itinerary_reliability(primary_itinerary,Backups,start_time,time_budget)
        if reliability > MRIB_reliability:
            MRIB_reliability = reliability
            MRIB = primary_itinerary
            MRIB_Backups = Backups
    print(transfer_dict)
    return MRIB_reliability, MRIB, MRIB_Backups

In [43]:
origin_node = "Schattendorf Kirchengasse" # for now the departure node is 1
destination_node = "Flughafen Wien Bahnhof"
start_time = "2024-12-12 14:30:00"
time_budget = timedelta(hours=2, minutes = 10)

MRIB_reliability, MRIB, Backups = find_path(origin_node,destination_node,start_time,time_budget)

1900-01-01 16:40:00
Network 18495
New iteration 1
New iteration 2
New iteration 3
New iteration 4
New iteration 5
New iteration 6
New iteration 7
New iteration 8
New iteration 9
New iteration 10
New iteration 11
New iteration 12
New iteration 13
New iteration 14
New iteration 15
New iteration 16
New iteration 17
New iteration 18
New iteration 19
New iteration 20
New iteration 21
New iteration 22
New iteration 23
New iteration 24
New iteration 25
New iteration 26
New iteration 27
New iteration 28
New iteration 29
New iteration 30
New iteration 31
New iteration 32
New iteration 33
New iteration 34
New iteration 35
New iteration 36
New iteration 37
New iteration 38
New iteration 39
New iteration 40
New iteration 41
New iteration 42
New iteration 43
New iteration 44
New iteration 45
New iteration 46
New iteration 47
New iteration 48
New iteration 49
New iteration 50
New iteration 51
New iteration 52
New iteration 53
New iteration 54
New iteration 55
New iteration 56
New iteration 57
New it

KeyboardInterrupt: 

In [36]:
print(MRIB_reliability, MRIB)
for i in Backups:
    print(i)

1.2536828019103206 [('150.TA.20-SV5-L-j24-1.13.H', 'Schattendorf Kirchengasse', '14:43:00', 'Loipersbach-Schattendorf Bhf.', '14:48:00', '20-SV5-L-j24-1', 'TA+byk20'), ('42.TA.3-R93-A-j24-1.7.R', 'Loipersbach-Schattendorf Bhf.', '14:54:00', 'Marz-Rohrbach Bahnhof', '14:58:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Marz-Rohrbach Bahnhof', '14:59:00', 'Mattersburg Bahnhof', '15:01:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Mattersburg Bahnhof', '15:02:00', 'Mattersburg Nord Bahnhof', '15:03:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Mattersburg Nord Bahnhof', '15:04:00', 'Wiesen-Sigleß Bahnhof', '15:07:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Wiesen-Sigleß Bahnhof', '15:07:00', 'Bad Sauerbrunn Bahnhof', '15:11:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Bad Sauerbrunn Bahnhof', '15:13:00', 'Neudörfl Bahnhof', '15:16:00', '3-R93-A-j24-1', 'TA+3j030'), ('42.TA.3-R93-A-j24-1.7.R', 'Neud

In [37]:
print(transform_route_info(MRIB,MRIB_reliability,Backups))

  🚆 Schattendorf Kirchengasse (Abfahrt: 14:43:00) → Loipersbach-Schattendorf Bhf. (Ankunft: 14:48:00) mit Linie 20-SV5-L-j24-1
  🚆 Loipersbach-Schattendorf Bhf. (Abfahrt: 14:54:00) → Marz-Rohrbach Bahnhof (Ankunft: 14:58:00) → Mattersburg Bahnhof (Ankunft: 15:01:00) → Mattersburg Nord Bahnhof (Ankunft: 15:03:00) → Wiesen-Sigleß Bahnhof (Ankunft: 15:07:00) → Bad Sauerbrunn Bahnhof (Ankunft: 15:11:00) → Neudörfl Bahnhof (Ankunft: 15:16:00) → Katzelsdorf Bahnhof (Ankunft: 15:20:00) → Wiener Neustadt Hauptbahnhof (Ankunft: 15:25:00) mit Linie 3-R93-A-j24-1
  🚆 Wiener Neustadt Hauptbahnhof (Abfahrt: 15:27:00) → Wien Meidling (Ankunft: 15:55:00) mit Linie 10-A12-j24-1
  🚆 Wien Meidling (Abfahrt: 15:59:00) → Wien Matzleinsdorfer Platz (Ankunft: 16:01:00) → Wien Hauptbahnhof (Ankunft: 16:03:00) mit Linie 2-RX2-W-j24-1
  🚆 Wien Hauptbahnhof (Abfahrt: 16:09:00) → Flughafen Wien Bahnhof (Ankunft: 16:24:00) mit Linie 10-A3-j24-1

🎯 Endstation: Flughafen Wien Bahnhof (Ankunft: 16:24:00)
🔹 Gesamt-Zu